In [1]:
from string import ascii_uppercase
from datetime import datetime
from random import randint

import sqlite3
import pandas as pd

In [2]:
con = sqlite3.connect('flights.db')
cur = con.cursor()

### getting tables in database

In [3]:
table_names_query = "select name from sqlite_master where type='table';"
cur.execute(table_names_query)
table_names = cur.fetchall()
table_names = [table_name[0] for table_name in table_names]
table_names

['airports', 'airlines', 'routes']

In [4]:
for name in table_names:
    query = f"select count(*) from {name}"
    cur.execute(query)
    print(f"{name} {cur.fetchall()[0][0]}")

airports 8107
airlines 6048
routes 67663


### tables

In [5]:
get_airports_query = "select * from airports"
df = pd.read_sql_query(get_airports_query, con, "index")
df.head()

,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
index,,,,,,,,,,,,
0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


In [6]:
get_airlines_query = "select * from airlines"
df = pd.read_sql_query(get_airlines_query, con, "index")
df.head()

,id,name,alias,iata,icao,callsign,country,active
index,,,,,,,,
0,1,Private flight,\N,-,None,None,None,Y
1,2,135 Airways,\N,None,GNL,GENERAL,United States,N
2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
3,4,2 Sqn No 1 Elementary Flying Training School,\N,None,WYT,None,United Kingdom,N
4,5,213 Flight Unit,\N,None,TFU,None,Russia,N


In [7]:
get_airlines_query = "select * from airlines where active = 'Y'"
df = pd.read_sql_query(get_airlines_query, con, "index")
df.head()

,id,name,alias,iata,icao,callsign,country,active
index,,,,,,,,
0,1,Private flight,\N,-,None,None,None,Y
2,3,1Time Airline,\N,1T,RNX,NEXTIME,South Africa,Y
9,10,40-Mile Air,\N,Q5,MLA,MILE-AIR,United States,Y
12,13,Ansett Australia,\N,AN,AAA,ANSETT,Australia,Y
13,14,Abacus International,\N,1B,None,None,Singapore,Y


### insert row

In [8]:
# avoid string format due to SQL injection
value = "(6048, 19846, 'Test flight', '', '', null, null, null, 'Y')"
insert_query = f"insert into airlines values {value}"
cur.execute(insert_query)
con.commit()

In [9]:
cur.execute("select count(*) from airlines")
assert cur.fetchall()[0][0] == 6049

In [10]:
pd.read_sql_query("select * from airlines where id=19846", con)

,index,id,name,alias,iata,icao,callsign,country,active
0,6048,19846,Test flight,,,None,None,None,Y


### modify row

In [11]:
values = ('USA', 19846)
mod_query = "update airlines set country=? where id=?"
cur.execute(mod_query, values)
con.commit()

In [12]:
df = pd.read_sql_query("select * from airlines", con, "index")

In [13]:
df.iloc[6048, 6] == "USA"

True

### delete row

In [14]:
values = (19846,)
delete_query = "delete from airlines where id=?"
cur.execute(delete_query, values)
con.commit()

In [15]:
cur.execute("select count(*) from airlines")
assert cur.fetchall()[0][0] == 6048

### create table

#### like

In [16]:
like_query = "create table airlines_like like airlines"
con.execute(like_query)
con.commit()

OperationalError: near "like": syntax error

#### clone

In [17]:
clone_query = "create table airlines_clone clone airlines"
con.execute(clone_query)
con.commit()

OperationalError: near "clone": syntax error

#### using connection

In [18]:
create_query = "create table daily_flights (id integer, departure date, arrival date, number text, route_id integer)"
con.execute(create_query)
con.commit()

In [19]:
insert_query = "insert into daily_flights values (1, '2016-09-28 0:00', '2016-09-28 12:00', 'T1', 1)"
con.execute(insert_query)
con.commit()

In [20]:
cur.execute("select count(*) from daily_flights")
assert cur.fetchall()[0][0] == 1

In [21]:
pd.read_sql_query("select * from daily_flights", con)

,id,departure,arrival,number,route_id
0,1,2016-09-28 0:00,2016-09-28 12:00,T1,1


#### using pandas

In [22]:
columns = ["id", "departure", "arrival", "number", "route_id"]
rows = [[1, datetime(2016, 9, 29, 0, 0), datetime(2016, 9, 29, 12, 0), 'T1', 1]]
df = pd.DataFrame(rows, columns=columns)

In [23]:
df.to_sql("daily_flights", con, if_exists="replace", index=False)

In [24]:
df = pd.read_sql_query("select count(*) from daily_flights", con)
assert len(df) == 1

### alter table

#### using connection

In [25]:
alter_query = "alter table daily_flights add column capacity integer"
con.execute(alter_query)

In [26]:
df = pd.read_sql_query("select * from daily_flights", con)
assert list(df.columns) == columns + ['capacity']

#### using pandas

In [27]:
df = pd.read_sql("select * from daily_flights", con)
df['delay'] = None
df.to_sql('daily_flights', con, if_exists='replace', index=False)

In [28]:
df = pd.read_sql_query("select * from daily_flights", con)
assert list(df.columns) == columns + ['capacity', 'delay']

### more tables

In [29]:
n  = 10

In [30]:
rows = []
for i in range(n):
    row = [i, ascii_uppercase[i], f"address_{i}", 18 + i / 10]
    rows.append(row)
    
columns = ['passenger_id', 'name', 'address', 'age']
df_passengers = pd.DataFrame(rows, columns=columns)
df_passengers

,passenger_id,name,address,age
0,0,A,address_0,18.0
1,1,B,address_1,18.1
2,2,C,address_2,18.2
3,3,D,address_3,18.3
4,4,E,address_4,18.4
5,5,F,address_5,18.5
6,6,G,address_6,18.6
7,7,H,address_7,18.7
8,8,I,address_8,18.8
9,9,J,address_9,18.9


In [31]:
df_passengers.to_sql('passengers', con, if_exists='replace', index=False)

In [32]:
rows = []
for i in [0, 2, 4, 6, 8]:
    row = [i, randint(100, 110)]
    rows.append(row)

columns = ['passenger_id', 'flight_id']
df_flights = pd.DataFrame(rows, columns=columns)
df_flights

,passenger_id,flight_id
0,0,102
1,2,106
2,4,109
3,6,102
4,8,110


In [33]:
df_flights.to_sql('flights', con, if_exists='replace', index=False)

### order by

In [34]:
order_query = "select * from passengers order by passengers.name desc"
pd.read_sql_query(order_query, con)

,passenger_id,name,address,age
0,9,J,address_9,18.9
1,8,I,address_8,18.8
2,7,H,address_7,18.7
3,6,G,address_6,18.6
4,5,F,address_5,18.5
5,4,E,address_4,18.4
6,3,D,address_3,18.3
7,2,C,address_2,18.2
8,1,B,address_1,18.1
9,0,A,address_0,18.0


### group by

In [35]:
group_by_query = "select avg(passenger_id) as average, flight_id from flights group by flight_id"
pd.read_sql_query(group_by_query, con)

,average,flight_id
0,3.0,102
1,2.0,106
2,4.0,109
3,8.0,110


### join

#### inner join

In [36]:
query = "select flights.flight_id, passengers.name, passengers.age from passengers inner join flights on flights.passenger_id = passengers.passenger_id"
pd.read_sql_query(query, con)

,flight_id,name,age
0,102,A,18.0
1,106,C,18.2
2,109,E,18.4
3,102,G,18.6
4,110,I,18.8


#### left join

In [37]:
query = "select flights.flight_id, passengers.name, passengers.age from passengers left join flights on flights.passenger_id = passengers.passenger_id"
pd.read_sql_query(query, con)

,flight_id,name,age
0,102.0,A,18.0
1,NaN,B,18.1
2,106.0,C,18.2
3,NaN,D,18.3
4,109.0,E,18.4
5,NaN,F,18.5
6,102.0,G,18.6
7,NaN,H,18.7
8,110.0,I,18.8
9,NaN,J,18.9


#### right join
not supported</br>
switch tables and use left join

In [38]:
query = "select flights.flight_id, passengers.name, passengers.age from flights left join passengers on flights.passenger_id = passengers.passenger_id"
pd.read_sql_query(query, con)

,flight_id,name,age
0,102,A,18.0
1,106,C,18.2
2,109,E,18.4
3,102,G,18.6
4,110,I,18.8


### drop tables

In [39]:
con.execute("drop table daily_flights")
con.execute("drop table passengers")
con.execute("drop table flights")
con.commit()

In [40]:
cur.execute(table_names_query)
names = cur.fetchall()
names = [name[0] for name in names]
assert names == table_names